In [134]:
using LinearAlgebra
using Flux

function Df(model, x)
    # x: single datapoint
    m = length(model(x))
    k = sum(length, Flux.params(model)) # Total amount of params

    # Þetta anonymous function reiknar gradient fyrir hvert function í outputinu frá 1:m
    jac = (fi) -> Flux.jacobian(() -> model(x)[fi],Flux.params(model)) # anonymous function

    # Skilgreini tómt Jacobian fylki
    Jacob = zeros(k,m)

    for func_i = 1:m
        current_col = []
        for param_i = 1:length(Flux.params(model))
            push!(current_col, jac(func_i)[Flux.params(model)[param_i]]) # Fyrir hvern parametra W1, B1, W2...
        end
        current_col = collect(Iterators.flatten(current_col)) # Flatten, flet allt
        # --- Spurning hvort hægt sé að gera þetta skilvirkara?
        # --- Held samt að Iterators pakkinn eigi að vera nokkuð skilvirkur

        Jacob[:, func_i] .= current_col # geri current_col að næsta dálka vigri jacobian
    end

    return Jacob # Þetta er Df fylkið í bilblíunni
end

function check_dim(x)
    """This function checks the appropriate  dimensions of input data"""
    if isa(x, Matrix)
        return size(x, 2)  # Returns the number of columns (width) of the matrix
    elseif isa(x, Vector)
        return 1  # Return 1 if it's a column vector
    else
        type = typeof(x)
        error("Input data type: $type is neither a matrix or column vector")
    end
end

function kernel(model, x)
    N = check_dim(x)
    m = length(model(x[:,1]))  # Number of functions in the model output
    K = zeros(N*m, N*m)
    
    for i = 1:N
        for j = 1:N
            block = Df(model, x[:,i])' * Df(model, x[:,j])
            K[(i-1)*m+1:i*m, (j-1)*m+1:j*m] .= block
        end
    end

    return K
end

kernel (generic function with 2 methods)

In [138]:
include("NN.jl")

X,Y, x_test, y_test = load_MNIST();

model = Chain(
    Dense(28*28, 60, sigmoid), # Input Layer -> Hidden Layer 1
    Dense(60, 60, sigmoid),     # Hidden Layer 1 -> Hidden Layer 2
    Dense(60, 10, sigmoid)      # Hidden Layer 2 -> Output Layer
)


K = kernel(model, X[:,1:10])

findall(a -> a<0, eigen.values(K))

100×100 Matrix{Float64}:
  1.14519       0.0195826     0.000760703  …   0.00312982   -0.0138963
  0.0195826     1.11551      -0.00626638      -0.000412202  -0.00277955
  0.000760703  -0.00626638    0.926612         0.00205764   -0.000799076
 -0.0155363     0.0114518     0.0101496       -0.00354303    0.00590882
 -0.025051      0.0165554    -0.00323857       0.0173471    -0.0185313
 -0.0208373     8.9945e-5    -0.0091585    …  -0.0061082     0.00567906
 -0.0029141    -0.0219589    -0.0100809       -0.00102331    0.00147196
  0.0186376     0.0389004    -0.0014876       -0.0208961    -0.0201299
  0.00531407    0.00359078   -0.00272739       1.10417      -0.0118959
 -0.0208827    -0.00796372   -0.00247641      -0.011918      0.957066
  ⋮                                        ⋱                
  0.0110082     1.10174      -0.0029721        0.00288373   -0.00598799
  0.00167476   -0.00298263    0.918235        -0.00107317   -0.00129064
 -0.0132107     0.00996397    0.00627876      -0.002743